In [ ]:
import sys
from pathlib import Path
import os
import sqlite3
import ipywidgets as widgets
from IPython.display import display
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
from dateutil.rrule import rrule, DAILY, MONTHLY
from dateutil.relativedelta import relativedelta

from trader.utils import ShioajiAccount
from trader.data import (Data, SQLiteTools, TickDBManager)
from trader.crawler import (Crawler, CrawlStockChip, CrawlStockTick, CrawlQuantX)
from trader.config import QUANTX_DB_PATH

### Load Data Crawlers

In [ ]:
try:
    crawler = Crawler()
    chip_crawler = crawler.chip
    tick_crawler = crawler.tick
    conn = sqlite3.connect(QUANTX_DB_PATH)
    qx_crawler = crawler.quantx
    
except Exception as e: # DolphinDB isn't connected
    chip_crawler = CrawlStockChip()
    tick_crawler = None
    conn = sqlite3.connect(QUANTX_DB_PATH)
    qx_crawler = CrawlQuantX()

### Update Chip Database

In [ ]:
chip_crawler.widget()

### Update Tick Database

#### 目前資料
* 2020/04/01 ~ 2024/05/10

In [ ]:
if tick_crawler is not None:
    tick_crawler.widget()

### Update QuantX Database

#### 每季財報 時間區間補充：

關於每季財報爬取，選取的時間範圍要「包含」以下時段，就能夠爬到截止日期的財報囉！例如：

* 2010年3月31號 --> 2009 第四季
* 2010年5月15號 --> 2010 第一季
* 2010年8月14號 --> 2010 第二季
* 2010年11月14號 --> 2010 第三季

所以假如我們想要抓2018年的第一季，就可以輸入一個時間範圍，例如：

* 2018年5月01號 到 2018年5月31號

因為這段時間有包含 2018年5月15號，所以就會下載 2018年第一季的財報喔！

In [ ]:
qx_crawler.widget(conn, 'price', qx_crawler.crawl_price, qx_crawler.date_range)
qx_crawler.widget(conn, 'monthly_revenue', qx_crawler.crawl_monthly_report, qx_crawler.month_range)
qx_crawler.widget(conn, 'finance_statement', qx_crawler.crawl_finance_statement_by_date, qx_crawler.season_range)
